In [ ]:
!git clone https://github.com/Jacobiano/morpholayers.git

In [ ]:
!wget -cO - "https://drive.google.com/uc?export=download&id=1pOYd-CQABj6vOkb76CImXGOJ6AGJqii9" > additiveShiftTutorialData.tar.gz
!tar -xzf additiveShiftTutorialData.tar.gz

In [ ]:
!ls

In [ ]:
!pip install scikit-image==0.18

In [ ]:
import skimage

In [ ]:
skimage.__version__

In [ ]:
import tensorflow as tf
import numpy as np

from morpholayers.layers import (
    Dilation2D,
    TopHatOpening2D,
    TopHatClosing2D,
    Gradient2D,
)

from tensorflow.keras import layers

In [ ]:
tf.__version__

In [ ]:
import matplotlib.pyplot as plt

plt.gray()

In [ ]:
from skimage import data
from skimage.color import rgb2gray

In [ ]:
im = rgb2gray(data.cat())

In [ ]:
plt.imshow(im, vmin=0.0, vmax=1.0)

In [ ]:
im_shifted = im + 0.3

In [ ]:
plt.imshow(im_shifted, vmin=0.0, vmax=1.0)

In [ ]:
# reshape to the format (B, H, W, C)
im = im[np.newaxis, ..., np.newaxis]
im_shifted = im_shifted[np.newaxis, ..., np.newaxis]

In [ ]:
# create a model consisting of the difference of a dilation and the input
inputs = layers.Input((None, None, 1))
x = Dilation2D(1, (11, 11))(inputs)
out = layers.Subtract()([x, inputs])
model = tf.keras.models.Model(inputs, out)

In [ ]:
# compute the operator on the input image
im_out = model(im)[0, :, :, 0]

plt.imshow(im_out)

In [ ]:
# shift the image by some amount and then compute the operator on the new image
im_out2 = model(im_shifted)[0, :, :, 0]
plt.imshow(im_out2)

In [ ]:
# compute the maximum difference between the two images
dist = float(tf.reduce_max(tf.abs(im_out - im_out2)))
print(dist)

In [ ]:
# create a top-hat operator:
# Image - Opening
inputs = layers.Input((None, None, 1))
out = TopHatOpening2D(1, (15, 15))(inputs)
model = tf.keras.models.Model(inputs, out)

In [ ]:
# compute the operator on the input image
im_out = model(im)[0, :, :, 0]

plt.imshow(im_out)

In [ ]:
# shift the image by some amount and then compute the operator on the new image
im_out2 = model(im_shifted)[0, :, :, 0]

plt.imshow(im_out2)

In [ ]:
# compute the maximum difference between the two images
dist = float(tf.reduce_max(tf.abs(im_out - im_out2)))
print(dist)

In [ ]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.fashion_mnist.load_data()

In [ ]:
x_train = x_train.astype(np.float32) / 255
x_test = x_test.astype(np.float32) / 255

In [ ]:
plt.figure()
plt.imshow(x_train[0, ...])
plt.figure()
plt.imshow(x_train[123, ...])
plt.figure()
plt.imshow(x_train[1234, ...])

In [ ]:
# define a model where the first layer is additive shift-invariant
# the model itself will be invariant to additive shift
inputs = layers.Input((28, 28, 1))
tho = TopHatOpening2D(2, (5, 5))(inputs)
thc = TopHatClosing2D(2, (5, 5))(inputs)
grad = Gradient2D(2, (5, 5))(inputs)

x0 = layers.Concatenate(-1)([tho, thc, grad])

x = layers.Conv2D(16, (3, 3), padding="same")(x0)
x = layers.BatchNormalization()(x)
x = layers.Activation("relu")(x)
x = layers.MaxPool2D()(x)
x = layers.Conv2D(32, (3, 3), padding="same")(x)
x = layers.BatchNormalization()(x)
x = layers.Activation("relu")(x)

x = layers.GlobalMaxPooling2D()(x)
x = layers.Dense(10, activation="softmax")(x)

model_invariant = tf.keras.models.Model(inputs, x)
model_invariant.summary()

In [ ]:
model_invariant.compile(
    loss=tf.keras.losses.SparseCategoricalCrossentropy(),
    optimizer=tf.keras.optimizers.Adam(1e-2),
    metrics=["accuracy"],
)

In [ ]:
model_invariant = tf.keras.models.load_model("additiveShiftTutorial_invariantModel")

In [ ]:
# model_invariant.fit(x_train, y_train, batch_size=64, epochs=10, validation_data=(x_test, y_test))

In [ ]:
# create a non-invariant model to use as a baseline
inputs = layers.Input((28, 28, 1))


x0 = layers.Conv2D(6, (5, 5), padding="same")(inputs)

x = layers.Conv2D(16, (3, 3), padding="same")(x0)
x = layers.BatchNormalization()(x)
x = layers.Activation("relu")(x)
x = layers.MaxPool2D()(x)
x = layers.Conv2D(32, (3, 3), padding="same")(x)
x = layers.BatchNormalization()(x)
x = layers.Activation("relu")(x)

x = layers.GlobalMaxPooling2D()(x)
x = layers.Dense(10, activation="softmax")(x)

model_noninvariant = tf.keras.models.Model(inputs, x)
model_noninvariant.summary()
model_noninvariant.compile(
    loss=tf.keras.losses.SparseCategoricalCrossentropy(),
    optimizer=tf.keras.optimizers.Adam(1e-2),
    metrics=["accuracy"],
)

In [ ]:
# model_noninvariant.fit(x_train, y_train, batch_size=64, epochs=10, validation_data=(x_test, y_test))

In [ ]:
model_noninvariant = tf.keras.models.load_model("additiveShiftTutorial_baselineModel")

In [ ]:
# model_invariant.save('additiveShiftTutorial_invariantModel')
# model_noninvariant.save('additiveShiftTutorial_baselineModel')

In [ ]:
print("evaluating the non additive shift invariant model")
model_noninvariant.evaluate(x_test, y_test)

In [ ]:
print("evaluating the additive shift invariant model")
model_invariant.evaluate(x_test, y_test)

In [ ]:
# create a new dataset by randomly shifting the Fashion MNIST test images
x_test2 = x_test + np.random.uniform(-1.0, 1.0, [x_test.shape[0], 1, 1])

In [ ]:
# show some examples of images from the new dataset
plt.figure(figsize=(10, 10))
plt.subplot(321)
plt.imshow(x_test[0, ...], vmin=0.0, vmax=1.0)
plt.title("Original Image")
plt.subplot(322)
plt.imshow(x_test2[0, ...], vmin=0, vmax=1.0)
plt.title("Shifted Image")
plt.subplot(323)
plt.imshow(x_test[1, ...], vmin=0.0, vmax=1.0)
plt.title("Original Image")
plt.subplot(324)
plt.imshow(x_test2[1, ...], vmin=0, vmax=1.0)
plt.title("Shifted Image")
plt.subplot(325)
plt.imshow(x_test[2, ...], vmin=0.0, vmax=1.0)
plt.title("Original Image")
plt.subplot(326)
plt.imshow(x_test2[2, ...], vmin=0, vmax=1.0)
plt.title("Shifted Image")

In [ ]:
# evaluate the non-invariant model on the new dataset
print("evaluating the non additive shift invariant model in the shifted test set")
model_noninvariant.evaluate(x_test2, y_test)

In [ ]:
# evaluate the invariant model in the new dataset
print("evaluating the additive shift invariant model in the shifted test set")
model_invariant.evaluate(x_test2, y_test)

In [ ]:
w1 = model_invariant.layers[1].weights[0]
plt.figure()
plt.subplot(121)
plt.imshow(w1[:, :, 0, 0])
plt.title("Closing top hat weights")
plt.subplot(122)
plt.imshow(w1[:, :, 0, 1])

In [ ]:
w2 = model_invariant.layers[2].weights[0]
plt.figure()
plt.subplot(121)
plt.imshow(w2[:, :, 0, 0])
plt.title("Closing top hat weights")
plt.subplot(122)
plt.imshow(w2[:, :, 0, 1])

In [ ]:
w3 = model_invariant.layers[3].weights[0]
plt.figure()
plt.subplot(121)
plt.imshow(w3[:, :, 0, 0])
plt.title("Gradient weights")
plt.subplot(122)
plt.imshow(w3[:, :, 0, 1])

In [ ]:
import tensorflow.keras.backend as K

operators = K.function(
    [model_invariant.layers[0].input], [model_invariant.layers[4].output]
)

In [ ]:
out_test = np.array(operators(x_test[:2, ...]))

In [ ]:
out_test.shape

In [ ]:
for i in range(out_test.shape[3]):
    plt.figure()
    plt.imshow(out_test[0, 0, :, :, i])